In [1]:
from experimental_chromatic_fitting import *
from chromatic import Rainbow
from pymc3 import Normal, Uniform, Model, HalfNormal,  plot_trace, sample_prior_predictive, sample_posterior_predictive
from tqdm import tqdm
from pymc3_ext import eval_in_model, optimize, sample
from exoplanet import QuadLimbDark, ImpactParameter

plt.matplotlib.style.use('default')

Running on Python v3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:23:20) 
[Clang 12.0.1 ]
Running on numpy v1.22.3
Running on chromatic v0.1.0
Running on PyMC v3.11.2
Running on ArviZ v0.12.0
Running on Exoplanet v0.5.2


In [ ]:
## load Patricio's spectrum model:
model_staticld, planet_params, wavelength,transmission = import_patricio_model()

# add wavelength-dep limb-darkening coeffs:
model_ld = add_ld_coeffs(model_staticld, planet_params, wavelength,transmission,mode="NIRCam_F322W2",star_params = {"M_H": -0.03, "Teff": 5326.6, "logg": 4.38933},plot=True)

# return synthetic rainbow + rainbow with injected transit:
bintime=5./60
binwave=0.15

# calculate the semi-major axis in stellar radii from the value in AU
# a_AU = 0.0486 # Faedi et al. 2011
# solar_radius_AU = 0.00465047
# stellar_radius = 0.895
# a_stellar_radii = a_AU/(solar_radius_AU * stellar_radius)

injplanet_params = {
            "t0": 0,
            "per": 4.055259, # days, Faedi et al. 2011
            "a": 11.37, # stellar radii
            "inc": 87.83, # degrees, Faedi et al. 2011
            "ecc": 0,
            "w": 0,
#             "limb_dark": "nonlinear",
#             "u": [0.0,0.2379,0.0,0.4615],
        }
# injplanet_params = {}#{'per':2}
r,i = inject_spectrum(model_ld, snr=1000, dt=bintime, res=50, planet_params=injplanet_params)

# bin in time and wavelength (to speed up fitting)
b_withouttransit = r.bin(nwavelengths=15,
                         dt=bintime * u.hour)#dw=binwave * u.micron,
b_withtransit = i.bin(nwavelengths=15,
                      dt=bintime * u.hour)
b_withtransit.save("test/b_withtransit_nw15.rainbow.npy")

In [ ]:
# load simulated rainbow:
b_withtransit = Rainbow("test/b_withtransit_dw015.rainbow.npy")
# load Eureka! rainbow:
# b_withtransit = Rainbow("ERS_eureka/S3_nircam_wfss_template_ap8_bg16_Table_Save.txt")
# load jwst rainbow:
# b_withtransit = Rainbow("ERS_jwst/*x1dints.fits")

In [ ]:
b_withtransit.imshow();
b_withtransit.imshow_quantities();
# b_withtransit.save("test/b_withtransit.rainbow.npy")

In [ ]:
bb = b_withtransit._create_copy()
# .bin(nwavelengths=b_withtransit.nwave)
# bb = bb._create_copy()

In [ ]:
bb.plot()

In [ ]:
t = TransitModel()

In [ ]:
t.initialize_empty_model()

# If I call something r_star instead of stellar_radius I will get two variables
t.setup_parameters(
#      period=Fitted(Uniform,testval=4.055259,lower=0.000009),
                  period=4.055259,
#                    t0=Fitted(Normal,mu=0.0,sigma=0.05),
                   epoch=Fitted(Normal,mu=0.0,sigma=0.0025),
#                    stellar_radius=0.895,
#                    stellar_mass=0.93,
                   stellar_radius = Fitted(Uniform, lower=0.890, upper=0.9,testval=0.895),
                   stellar_mass =Fitted(Uniform, lower=0.9, upper=1,testval=0.93),
                   radius_ratio=WavelikeFitted(Normal, mu=0.15, sigma=0.05),
#                    b=Fitted(ImpactParameter,ror=0.15,testval=0.44),
                   impact_parameter=Fitted(ImpactParameter,ror=0.15,testval=0.44),
                   limb_darkening=WavelikeFitted(QuadLimbDark,testval=[0.05,0.35]),
                   baseline=WavelikeFitted(Normal,mu=1.0,sigma=0.0025))
t.summarize_parameters()

In [ ]:
t.setup_orbit()

In [ ]:
t.attach_data(bb)

In [ ]:
t.plot_orbit()

In [ ]:
print(t.model)

In [ ]:
t.setup_lightcurves()

In [ ]:
t.setup_likelihood()

In [ ]:
prior_predictive_trace = t.sample_prior()
# with t.model:
#     prior_predictive_trace = sample_prior_predictive(3)

In [ ]:
for i in range(3):
    flux_for_this_sample = np.array([prior_predictive_trace[f'wavelength_{w}_data'][i] for w in range(bb.nwave)])
    bb.fluxlike[f'prior-predictive-{i}'] = flux_for_this_sample
bb.imshow_quantities()
# plt.plot(bb.fluxlike[f'prior-predictive-{2}'][0]);

In [ ]:
with t.model:
    opt = optimize(start=t.model.test_point)
    opt = optimize(start=opt)
    trace = sample(start=opt,tune=8000, draws=12000, cores=12, chains=4)
    summary = az.summary(trace, round_to=7, fmt='wide')
    print(summary)

In [ ]:
with t.model:
    trace = sample(tune=8000, draws=12000, cores=12, chains=4)
    summary = az.summary(trace, round_to=7, fmt='wide')
    print(summary)

In [ ]:
vars_to_opt = [t.model[f"radius_ratio_w{w}"] for w in range(bb.nwave)]
# vars_to_opt.append([t.model[f"limb_darkening_w{w}"] for w in range(bb.nwave)])

with t.model:
    opt = optimize(start=t.model.test_point)
    opt = optimize(start=opt, vars=vars_to_opt)
    trace = sample(start=opt,tune=30000, draws=30000, cores=12, chains=4)

In [ ]:
with t.model:
    summary = az.summary(trace, round_to=7, fmt='wide')
    print(summary)

In [ ]:
posterior_predictive_trace = t.sample_posterior(trace,ndraws=50)

In [ ]:
for i in range(5):
    flux_for_this_sample = np.array([posterior_predictive_trace[f'wavelength_{w}_data'][i] for w in range(bb.nwave)])
    bb.fluxlike[f'posterior-predictive-{i}'] = flux_for_this_sample
    
#     plt.figure()
#     bb.plot()
#     plt.plot(bb.time,bb.fluxlike[f'posterior-predictive-{i}'][0]);
#     plt.show()
#     plt.close()
bb.imshow_quantities()

In [ ]:
offset = 0
fig, ax = plt.subplots(figsize=(7,10))
from matplotlib.pyplot import cm
color=cm.viridis(np.linspace(0,1,bb.nwave))
firstplot = True
for w in range(bb.nwave):
    for i in range(50):
        if firstplot:
            plt.plot(bb.time, offset + posterior_predictive_trace[f'wavelength_{w}_data'][i],color='gray',alpha=0.5,label='Posterior Sample\n (n=50)')
            firstplot=False
        else:
            plt.plot(bb.time, offset + posterior_predictive_trace[f'wavelength_{w}_data'][i],color='gray',alpha=0.5)
    plt.plot(bb.time, offset + bb.flux[w],linestyle="None", markersize=8, marker='.', color=color[w],label="%.1f microns"%bb.wavelength[w].to_value())
    offset = offset-0.04
plt.ylabel("Relative Flux + offset")
plt.xlabel("Time from Transit Epoch")
plt.legend(loc='upper right',fontsize=8)

In [ ]:
summary

In [ ]:
# summary
# for k,v in summary['mean'].items():
#     if "radius_r"
model_staticld, planet_params, wavelength,transmission = import_patricio_model()

plt.figure(figsize=(7,4))
plt.plot(wavelength,transmission*100,color='cyan',alpha=0.5,label='Spectrum Model')
for w in range(bb.nwave):
    rr = (summary['mean'][f"radius_ratio_w{w}"]**2)*100
    hdi3 = (summary['hdi_3%'][f"radius_ratio_w{w}"]**2)*100
    hdi97 = (summary['hdi_97%'][f"radius_ratio_w{w}"]**2)*100
    plt.plot(bb.wavelength[w],rr,'k.',markersize=6)
    plt.errorbar(bb.wavelength[w],rr, xerr=0.05*u.micron,
                 yerr=np.array([np.abs(hdi3-rr),np.abs(hdi97-rr)]).reshape((2,1)),c='k',alpha=0.5,capsize=3)

plt.ylabel("Transit Depth [%]")
plt.xlabel("Wavelength [microns]")
plt.xlim(2,4.5)
plt.legend()
plt.show()

In [ ]:
az.plot_trace(trace);

In [49]:
t.model.test_point

{'epoch': array(0.),
 'impact_parameter_impact__': array(-0.47849024),
 'stellar_radius_interval__': array(-0.3292585),
 'stellar_mass_interval__': array(-0.25398823),
 'limb_darkening_w0_quadlimbdark__': array([-1.65822808, -2.7080502 ]),
 'radius_ratio_w0': array(0.15),
 'baseline_w0': array(1.)}

In [50]:
t.model_chromatic_flux

[Elemwise{add,no_inplace}.0]

In [89]:
t.model

In [171]:
print(t.model['wavelength_0_data'])

wavelength_0_data ~ Normal
